In [1]:
import torch
import csv
from torch.utils.data import dataset
import numpy as np
import os

In [3]:
train_path = "../data/training_data.csv"
train_filter_path = "../data/plan_data/filter_train3.csv"
min_max_path = "../data/column_min_max_vals.csv"
test_path = "../data/testing_data.csv"

In [4]:
# load data
def load_data(train_path, train_filter_path, train=True):
    tables = []
    joins = []
    conditions = []
    query_nums = []
    min_filter_num = np.inf
    max_filter_num = 0
    max_cost_num = 0
    min_cost_num = np.inf
#     lack = []
    with open(train_path, 'r') as f:
        with open(train_filter_path, 'r') as ff:
            data = csv.reader(f, delimiter = '#')
            data_filter = list(csv.reader(ff, delimiter = '#'))
            for i,row in enumerate(data):
                tables.append(row[0].split(','))
                joins.append(row[1].split(','))
                attrs = row[2].split(',')
                cond_vec = []
                filter_row = data_filter[i][1:-1]
                for j in range(0, len(attrs), 3):
                    if len(attrs) != 1:
                        cond_word = attrs[j].split('.')[-1] + ' ' + attrs[j+1] + ' ' + attrs[j+2]
#                         flag = 0
                        for k,item in enumerate(filter_row):
                            if item.startswith(cond_word):
                                flag = 1
#                                 print(item.split(',')[-1])
                                filter_num = int(item.split(',')[-2])
                                cost_num = int(item.split(',')[-1])
                                cond_vec.append(list(attrs[j: j+3]+[filter_num, cost_num]))
                                if filter_num > max_filter_num:
                                    max_filter_num = filter_num
                                elif filter_num < min_filter_num:
                                    min_filter_num = filter_num
                                if cost_num > max_cost_num:
                                    max_cost_num = cost_num
                                elif cost_num < min_cost_num:
                                    min_cost_num = cost_num
#                                 print(cond_vec)
#                         if flag == 0:
#                             print(cond_word)
#                             print(i)
#                             print("***********************OH NO***********************")
#                             lack.append(i)
                    else:
                        cond_vec.append(list(attrs[j: j+3]))
                conditions.append(cond_vec)
#                 conditions.append(list(attrs[i: i+3] for i in range(0, len(attrs), 3)))
#                 print(conditions[i])
                if train:
                    query_nums.append(int(row[3]))
#                 break
#     with open('lack.txt', 'w') as f:
#         for i in lack:
#             f.write(str(i))
#             f.write('\n')
    print(max_filter_num, min_filter_num)
    if train:
        return tables, joins, conditions, query_nums, max_filter_num, min_filter_num, max_cost_num, min_cost_num
    else:
        return tables, joins, conditions, max_filter_num, min_filter_num, max_cost_num, min_cost_num
tables, joins, conditions, query_nums, max_filter_num, min_filter_num,max_cost_num, min_cost_num = load_data(train_path, train_filter_path)

36244343 0


In [48]:
print(max_cost_num, min_cost_num)

36222066 1


In [45]:
conditions

[[['t.kind_id', '=', '7', 985048, 1540800],
  ['mi_idx.info_type_id', '>', '99', 459925, 919057]],
 [['t.production_year', '>', '2004', 1420387, 1107667]],
 [['mi.info_type_id', '<', '4', 11210743, 3623957]],
 [['mc.company_id', '<', '27', 2474322, 131143]],
 [['mk.keyword_id', '<', '55', 4469104, 55951]],
 [['t.production_year', '>', '1977', 551695, 1980005],
  ['mc.company_id', '>', '71403', 2040243, 564293],
  ['mi.info_type_id', '<', '4', 11210743, 3623957]],
 [['mk.keyword_id', '<', '35049', 516811, 3997001]],
 [['']],
 [['mi_idx.info_type_id', '>', '99', 459925, 919057]],
 [['mi.info_type_id', '>', '16', 12601870, 2194617],
  ['mi_idx.info_type_id', '=', '100', 920110, 454124]],
 [['ci.person_id', '=', '890821', 18122170, 176],
  ['ci.role_id', '=', '1', 18122170, 176],
  ['mk.keyword_id', '>', '3624', 2023483, 2505706]],
 [['t.production_year', '>', '2010', 2136646, 392896],
  ['mi.info_type_id', '<', '16', 5270569, 9476311]],
 [['t.production_year', '<', '2010', 605463, 1923403

In [5]:
def load_min_max(min_max_path):
    attr_min_max = {}
    with open(min_max_path, 'r') as f:
        data = csv.reader(f, delimiter=',')
        for i,row in enumerate(data):
            if i == 0:
                continue
            attr_min_max[row[0]] = [int(row[1]), int(row[2])]
    return attr_min_max
attr_min_max = load_min_max(min_max_path)
attr_min_max

{'t.id': [1, 2528312],
 't.kind_id': [1, 7],
 't.production_year': [1880, 2019],
 'mc.id': [1, 2609129],
 'mc.company_id': [1, 234997],
 'mc.movie_id': [2, 2525745],
 'mc.company_type_id': [1, 2],
 'ci.id': [1, 36244344],
 'ci.movie_id': [1, 2525975],
 'ci.person_id': [1, 4061926],
 'ci.role_id': [1, 11],
 'mi.id': [1, 14835720],
 'mi.movie_id': [1, 2526430],
 'mi.info_type_id': [1, 110],
 'mi_idx.id': [1, 1380035],
 'mi_idx.movie_id': [2, 2525793],
 'mi_idx.info_type_id': [99, 113],
 'mk.id': [1, 4523930],
 'mk.movie_id': [2, 2525971],
 'mk.keyword_id': [1, 134170]}

In [6]:
def load_plan(plan_path):
    max_row = 0
    max_time = 0
    min_row = np.inf
    min_time = np.inf
    plan_rows = []
    plan_time = []
    with open(plan_path, 'r') as f:
        data = csv.reader(f)
        for i,row in enumerate(data):
            if i==0:
                continue
            #print(row[0],row[1])
            cur_row = int(row[0])
            cur_time = float(row[1])
            plan_rows.append(cur_row)
            plan_time.append(cur_time)

#             if cur_time < 1:   没有小于1的时间，故可以使用Log处理
#                 print('Yes')
    min_row = min(plan_rows)
    max_row = max(plan_rows)
    min_time = min(plan_time)
    max_time = max(plan_time)
    return min_row, max_row, min_time, max_time, plan_rows, plan_time
min_train_plan_row, max_train_plan_row,min_train_plan_time, max_train_plan_time,train_plan_rows, train_plan_time = load_plan('plan_data/train.csv')
min_test_plan_row, max_test_plan_row,min_test_plan_time, max_test_plan_time,test_plan_rows, test_plan_time = load_plan('plan_data/test.csv')

In [7]:
len(train_plan_rows)

100000

In [7]:
def get_category(conditions, tables, joins):
    attr_names = set()
    operators = set()
    table_names = set()
    join_types = set()
    for row in conditions:
        for item in row:
            if len(item) == 5:
                attr_names.add(item[0])
                operators.add(item[1])
    for row in tables:
        for item in row:
            table_names.add(item)
    for row in joins:
        for item in row:
            join_types.add(item)
    return table_names, join_types, attr_names, operators
table_names, join_types, attr_names, operators = get_category(conditions, tables, joins)
print(join_types)
print(table_names)
print(operators)
print(attr_names)

{'', 't.id=mk.movie_id', 't.id=mi.movie_id', 't.id=mc.movie_id', 't.id=ci.movie_id', 't.id=mi_idx.movie_id'}
{'cast_info ci', 'title t', 'movie_info mi', 'movie_keyword mk', 'movie_info_idx mi_idx', 'movie_companies mc'}
{'=', '>', '<'}
{'mc.company_id', 'mc.company_type_id', 't.kind_id', 'mi.info_type_id', 'ci.person_id', 'ci.role_id', 'mi_idx.info_type_id', 'mk.keyword_id', 't.production_year'}


In [8]:
def one_hot_map(data_set):
    ele_num = len(data_set)
    data = list(data_set)
    data.sort()
    def one_hot_encode(idx):
        one_hot = np.zeros(ele_num)
        one_hot[idx] = 1
        return one_hot
    data_onehot_mapping = {item: one_hot_encode(i) for i,item in enumerate(data)}
    onehot_data_mapping = [item for item in data]
    return data_onehot_mapping, onehot_data_mapping
table2vec, table_list = one_hot_map(table_names)
attr2vec, attr_list = one_hot_map(attr_names)
oper2vec, oper_list = one_hot_map(operators)
join2vec, join_list = one_hot_map(join_types)
print(table2vec, table_list)
print(attr2vec, attr_list)
print(oper2vec, oper_list)
print(join2vec, join_list)

{'cast_info ci': array([1., 0., 0., 0., 0., 0.]), 'movie_companies mc': array([0., 1., 0., 0., 0., 0.]), 'movie_info mi': array([0., 0., 1., 0., 0., 0.]), 'movie_info_idx mi_idx': array([0., 0., 0., 1., 0., 0.]), 'movie_keyword mk': array([0., 0., 0., 0., 1., 0.]), 'title t': array([0., 0., 0., 0., 0., 1.])} ['cast_info ci', 'movie_companies mc', 'movie_info mi', 'movie_info_idx mi_idx', 'movie_keyword mk', 'title t']
{'ci.person_id': array([1., 0., 0., 0., 0., 0., 0., 0., 0.]), 'ci.role_id': array([0., 1., 0., 0., 0., 0., 0., 0., 0.]), 'mc.company_id': array([0., 0., 1., 0., 0., 0., 0., 0., 0.]), 'mc.company_type_id': array([0., 0., 0., 1., 0., 0., 0., 0., 0.]), 'mi.info_type_id': array([0., 0., 0., 0., 1., 0., 0., 0., 0.]), 'mi_idx.info_type_id': array([0., 0., 0., 0., 0., 1., 0., 0., 0.]), 'mk.keyword_id': array([0., 0., 0., 0., 0., 0., 1., 0., 0.]), 't.kind_id': array([0., 0., 0., 0., 0., 0., 0., 1., 0.]), 't.production_year': array([0., 0., 0., 0., 0., 0., 0., 0., 1.])} ['ci.perso

In [19]:
attr2vec

{'ci.person_id': array([1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'ci.role_id': array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 'mc.company_id': array([0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 'mc.company_type_id': array([0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 'mi.info_type_id': array([0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 'mi_idx.info_type_id': array([0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 'mk.keyword_id': array([0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 't.kind_id': array([0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 't.production_year': array([0., 0., 0., 0., 0., 0., 0., 0., 1.])}

In [9]:
def normalize_query_nums(query_nums):
    query_nums = np.log(np.array(query_nums) + 1)
    min_num = query_nums.min()
    max_num = query_nums.max()
    query_nums_norm = (query_nums - min_num) / (max_num - min_num)
    query_nums_norm = np.minimum(query_nums_norm, 1)
    query_nums_norm = np.maximum(query_nums_norm, 0)
    return min_num, max_num, query_nums_norm
min_query_num, max_query_num, query_nums_norm = normalize_query_nums(query_nums)
print(min_query_num, max_query_num, query_nums_norm )

0.6931471805599453 19.947728021487798 [0.61610916 0.68684191 0.74840433 ... 0.56385215 0.08358727 0.71433479]


In [10]:
# 属性归一化
def normalize_attr(val, attr_name, attr_min_max):
    min_val = attr_min_max[attr_name][0]
    max_val = attr_min_max[attr_name][1]
    val_norm = 0.0
    if max_val > min_val:
        val_norm = (float(val) - min_val) / (max_val - min_val)
    return np.array(val_norm)

In [11]:
def normalize_plan(data, min_val, max_val):
    _data = np.array(data)
#     print(_data.min(), _data.max())
    _data = np.log(_data)
    min_val = np.log(min_val)
    max_val = np.log(max_val)
    data_norm = (_data - min_val) / (max_val - min_val)
    data_norm = np.minimum(data_norm, 1)
    data_norm = np.maximum(data_norm, 0)
    return data_norm

In [12]:
def normalize(val, min_val, max_val):
    if val == 0:
        return 0
    _val = np.log(val)
    max_val = np.log(max_val)
    val_norm = (_val - min_val) / (max_val - min_val)
    val_norm = np.minimum(val_norm, 1)
    val_norm = np.maximum(val_norm, 0)
    return val_norm

In [37]:
np.minimum(2,1)

1

In [39]:
np.minimum(np.array([1,2,3]), 1)


array([1, 1, 1])

In [12]:
join2vec

{'': array([1., 0., 0., 0., 0., 0.]),
 't.id=ci.movie_id': array([0., 1., 0., 0., 0., 0.]),
 't.id=mc.movie_id': array([0., 0., 1., 0., 0., 0.]),
 't.id=mi.movie_id': array([0., 0., 0., 1., 0., 0.]),
 't.id=mi_idx.movie_id': array([0., 0., 0., 0., 1., 0.]),
 't.id=mk.movie_id': array([0., 0., 0., 0., 0., 1.])}

In [9]:
conditions

[[['t.kind_id', '=', '7'], ['mi_idx.info_type_id', '>', '99']],
 [['t.production_year', '>', '2004']],
 [['mi.info_type_id', '<', '4']],
 [['mc.company_id', '<', '27']],
 [['mk.keyword_id', '<', '55']],
 [['t.production_year', '>', '1977'],
  ['mc.company_id', '>', '71403'],
  ['mi.info_type_id', '<', '4']],
 [['mk.keyword_id', '<', '35049']],
 [['']],
 [['mi_idx.info_type_id', '>', '99']],
 [['mi.info_type_id', '>', '16'], ['mi_idx.info_type_id', '=', '100']],
 [['ci.person_id', '=', '890821'],
  ['ci.role_id', '=', '1'],
  ['mk.keyword_id', '>', '3624']],
 [['t.production_year', '>', '2010'], ['mi.info_type_id', '<', '16']],
 [['t.production_year', '<', '2010'], ['mi_idx.info_type_id', '=', '101']],
 [['ci.role_id', '=', '10']],
 [['t.kind_id', '>', '1']],
 [['mi.info_type_id', '=', '3']],
 [['t.production_year', '>', '2003'], ['mi.info_type_id', '=', '16']],
 [['mi.info_type_id', '>', '16']],
 [['mi.info_type_id', '<', '16']],
 [['mk.keyword_id', '=', '18559']],
 [['']],
 [['mi.info

In [13]:
def encode_vector(conditions, joins, attr_min_max, attr2vec, oper2vec,join2vec, min_filter_num, max_filter_num, min_cost_num, max_cost_num):
    conditions_encoding = []
    joins_encoding = []
    max_cond_num = 0
    max_join_num = 0
    for i,row in enumerate(conditions):
        cond_vec = []
        for item in row:
            if len(item) == 5:
                c_vec=[]
                val = normalize_attr(float(item[2]), item[0], attr_min_max)
                filter_num = normalize(item[3], min_filter_num, max_filter_num)
                cost_num = normalize(item[4], min_cost_num, max_cost_num)
                c_vec.append(attr2vec[item[0]])
                c_vec.append(oper2vec[item[1]])
                c_vec.append(val)
                c_vec.append(filter_num)
                c_vec.append(cost_num)
                c_vec = np.hstack(c_vec)
#                 print(c_vec)
            else:
                c_vec = np.zeros((len(attr2vec) + len(oper2vec) + 3)) # 无条件，填0
            cond_vec.append(c_vec)
        if max_cond_num < len(cond_vec):
            max_cond_num = len(cond_vec)
        conditions_encoding.append(cond_vec)
        
        join_vec = []
        for join in joins[i]:
            join_vec.append(join2vec[join])
        if max_join_num < len(join_vec):
            max_join_num = len(join_vec)
        joins_encoding.append(join_vec)
    print(max_cond_num)
    return conditions_encoding, joins_encoding, max_cond_num, max_join_num

In [14]:
def encoding(query_num, train_val_ratio):
    train_path = "../data/training_data.csv"
    min_max_path = "../data/column_min_max_vals.csv"
#     test_path = "../data/testing_data.csv"
    test_path = "../data/new_testing_data.csv"
    plan_train_path = "../data/plan_data/train.csv"
#     plan_test_path = "../data/plan_data/test.csv"
    plan_test_path = "../data/plan_data/new_test.csv"
    train_filter_path = "../data/plan_data/filter_train2.csv"
#     test_filter_path = "../data/plan_data/filter_test2.csv"
    test_filter_path = "../data/plan_data/filter_newtest2.csv"
    tables, joins, conditions, query_nums, max_filter_num1, min_filter_num1,max_cost_num1, min_cost_num1 = load_data(train_path, train_filter_path)
    test_tables, test_joins, test_conditions, max_filter_num2, min_filter_num2,max_cost_num2, min_cost_num2 = load_data(test_path, test_filter_path, train=False)
    min_train_plan_rows, max_train_plan_rows, min_train_plan_time, max_train_plan_time, train_plan_rows, train_plan_time = load_plan(plan_train_path)
    min_test_plan_rows, max_test_plan_rows, min_test_plan_time, max_test_plan_time, test_plan_rows, test_plan_time = load_plan(plan_test_path)

    #     print("*******************PLAN************************")
#     print(len(train_plan_rows),train_plan_rows[:5])
#     print(len(test_plan_rows), test_plan_rows[:5])
#     print(train_plan_time[:5])
#     print(test_plan_time[:5])
#     print(max_filter_num1, min_filter_num1)
#     print(max_cost_num1, min_cost_num1)

    min_filter_num = min(min_filter_num1, min_filter_num2)
    max_filter_num = max(max_filter_num1, max_filter_num2)
    min_cost_num = min(min_cost_num1, min_cost_num2)
    max_cost_num = max(max_cost_num1, max_cost_num2)
    min_plan_rows = min(min_train_plan_rows, min_test_plan_rows)
    max_plan_rows = max(max_train_plan_rows, max_test_plan_rows)
    min_plan_time = min(min_train_plan_time, min_test_plan_time)
    max_plan_time = max(max_train_plan_time, max_test_plan_time)
#     print(min_plan_rows, max_plan_rows)
#     print(min_plan_time, max_plan_time)
    
    table_names, join_types, attr_names, operators = get_category(conditions, tables, joins)
#     print("***************** JOIN *******************")
#     print(join_types)
#     print("***************** TABLE *******************")
#     print(table_names)
#     print("***************** OPERATORS *******************")
#     print(operators)
#     print("***************** ATTRIBUTES *******************")
#     print(attr_names)
    table2vec, table_list = one_hot_map(table_names)
    attr2vec, attr_list = one_hot_map(attr_names)
    oper2vec, oper_list = one_hot_map(operators)
    join2vec, join_list = one_hot_map(join_types)
#     print("***************** ONE-HOT *******************")
#     print(table2vec, table_list)
#     print(attr2vec, attr_list)
#     print(oper2vec, oper_list)
#     print(join2vec, join_list)
    attr_min_max = load_min_max(min_max_path)
#     print("***************** ATTR-MIN-MAX *******************")
#     print(attr_min_max)
    min_query_num, max_query_num, query_nums_norm = normalize_query_nums(query_nums)
    
    # plan data
    train_plan_rows_norm = normalize_plan(train_plan_rows, min_plan_rows, max_plan_rows)
    train_plan_time_norm = normalize_plan(train_plan_time, min_plan_time, max_plan_time)
    test_plan_rows_norm = normalize_plan(test_plan_rows, min_plan_rows, max_plan_rows)
    test_plan_time_norm = normalize_plan(test_plan_time, min_plan_time, max_plan_time)
#     print(train_plan_rows_norm[:5])
#     print(test_plan_rows_norm[:5])
#     print(train_plan_time_norm[:5])
#     print(test_plan_time_norm[:5])
    
#     print("***************** MIN-MAX-QUERY-NUM *******************")
#     print(min_query_num, max_query_num, query_nums_norm )
    conditions_encoding, joins_encoding, max_cond_num, max_join_num = encode_vector(conditions, joins, attr_min_max, attr2vec, oper2vec, join2vec, min_filter_num, max_filter_num, min_cost_num, max_cost_num)
#     print(conditions_encoding)
    test_conditions_encoding, test_joins_encoding, test_max_cond_num, test_max_join_num = encode_vector(test_conditions, test_joins, attr_min_max, attr2vec, oper2vec, join2vec, min_filter_num, max_filter_num,min_cost_num, max_cost_num)
#     print("***************** ENCODING *******************")
#     print(conditions_encoding[0])
#     print(joins_encoding[0])
#     print(test_conditions_encoding[0])
#     print(test_joins_encoding[0])
#     print("***************** max_cond_num & max_join_num *******************")
#     print(max_cond_num, max_join_num)
#     print(test_max_cond_num, test_max_join_num)
    
    # 划分训练集、验证集
    train_num = int(query_num * train_val_ratio)
    val_num = query_num - train_num
    
    train_conditions = conditions_encoding[:train_num]
    train_joins = joins_encoding[:train_num]
    train_query_num = query_nums_norm[:train_num]
    train_prows = train_plan_rows_norm[:train_num]
    train_ptime = train_plan_time_norm[:train_num]
    
    val_conditions = conditions_encoding[train_num: query_num]
    val_joins = joins_encoding[train_num: query_num]
    val_query_num = query_nums_norm[train_num: query_num]
    val_prows = train_plan_rows_norm[train_num: query_num]
    val_ptime = train_plan_time_norm[train_num: query_num]
    
    max_cond_num = max(max_cond_num, test_max_cond_num)
    max_join_num = max(max_join_num, test_max_join_num)
#     print(max_cond_num, max_join_num)
    
    dicts = [table2vec, attr2vec, oper2vec, join2vec]
#     print("***************** Dicts *******************")
#     print(dicts)
    train_data = [train_conditions, train_joins, train_prows, train_ptime]
    val_data = [val_conditions, val_joins, val_prows, val_ptime]
    test_data = [test_conditions_encoding, test_joins_encoding, test_plan_rows_norm, test_plan_time_norm]
#     print("***************** train_data *******************")
#     print(val_data)
#     print(train_data[2].shape)
    return train_data, val_data, test_data, train_query_num, val_query_num, min_query_num, max_query_num, dicts, attr_min_max, max_cond_num, max_join_num


In [15]:
# 构建训练数据 especially padding
def construct_data(conditions, joins, plan_rows, plan_time, max_join_num, max_cond_num):
    condition_vecs = []
    for cond in conditions:
        condition_vec = np.vstack(cond)
        num_pad = max_cond_num - condition_vec.shape[0]
        condition_vec = np.pad(condition_vec, ((0, num_pad), (0,0)), 'constant')
#         condition_vec = np.expand_dims(condition_vec, 0)
        condition_vecs.append(np.hstack(condition_vec))
    condition_vecs = np.vstack(condition_vecs)
#     print(condition_vecs[0])
#     print(condition_vecs[0].shape[0])
    
    join_vecs = []
    for join in joins:
        join_vec = np.vstack(join)
        num_pad = max_join_num - join_vec.shape[0]
        join_vec = np.pad(join_vec, ((0, num_pad), (0,0)), 'constant')
#         join_vec = np.expand_dims(join_vec, 0)
        join_vecs.append(np.hstack(join_vec))
    join_vecs = np.vstack(join_vecs)
#     print(len(join_vecs))
#     print(join_vecs[0])
#     print(join_vecs[0].shape[0])

    plan_rows_vecs = np.vstack(plan_rows)
    plan_time_vecs = np.vstack(plan_time)
    dataset = np.concatenate((condition_vecs, join_vecs, plan_rows_vecs, plan_time_vecs),axis=1)
#     dataset = np.concatenate((condition_vecs, join_vecs),axis=1)
#     print(dataset[0])
#     print(dataset[0].shape[0])
    return dataset

In [16]:
def get_datasets(query_num, train_val_ratio=0.9):
    train_data, val_data, test_data, train_query_num, val_query_num, min_query_num, max_query_num, dicts, attr_min_max,max_cond_num, max_join_num = encoding(query_num, train_val_ratio)
    train_dataset = construct_data(*train_data, max_join_num, max_cond_num)
    if train_val_ratio < 1.0:
        val_dataset = construct_data(*val_data, max_join_num, max_cond_num)
    else:
        val_dataset = []
    test_dataset = construct_data(*test_data, max_join_num, max_cond_num)
#     print(train_dataset.shape)
#     print(val_dataset.shape)
#     print(test_dataset.shape)
    return train_dataset, val_dataset, test_dataset, train_query_num, val_query_num, min_query_num, max_query_num, dicts, attr_min_max,max_cond_num, max_join_num
train_dataset, val_dataset, test_dataset, train_query_num, val_query_num, min_query_num, max_query_num, dicts, attr_min_max,max_cond_num, max_join_num = get_datasets(10000)

In [153]:
train_dataset[1].shape

(104,)

In [10]:
a = np.array([1,2,3])
np.vstack(a)

array([[1],
       [2],
       [3]])


# 模型训练

In [17]:
def denormalize_query_nums(y_pred, min_query_num, max_query_num):
    query_num_norm = np.array(y_pred, dtype=np.float32)
    query_num = (query_num_norm * (max_query_num - min_query_num)) + min_query_num
    return np.array(np.round(np.exp(query_num)-1), dtype=np.int64)

In [18]:
def RMSLE(y_test, y_pred):
    n = y_test.shape[0]
    return np.sqrt( np.sum((y_test - y_pred) * (y_test - y_pred)) / n)

In [19]:
# Train and Test models
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from scipy import interp
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mse
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import model_selection
from sklearn import ensemble

In [35]:
rmse = []

In [23]:
model_name = ['Random Forest', 'Adaboost', 'SVR','Bayesian Ridge', 'ElasticNet', 'GradientBoosting', 'MLP']

In [37]:
# Random Forest
def random_forest(X,X_test,y,y_test,validate=True):
    y_pred = []
    model_clf = ensemble.RandomForestRegressor(max_depth = None, max_features = 12, min_samples_split = 2, n_estimators = 100)
#     hyper_params = {'n_estimators':[50,100,150],'min_samples_split':[2,4,8], 'max_features':[8,10,12], 'max_depth':[None,1,3,5]}
#     clf = model_selection.GridSearchCV(model_clf, hyper_params,cv=3,verbose=0, n_jobs=20)
#     clf.fit(X,y)
#     y_pred = clf.predict(X_test)
    model_clf.fit(X,y)
    y_pred = model_clf.predict(X_test)
    if validate:
        print('Model 1. Random Forest\n')
#         print('Best-parameters:{}'.format(clf.best_params_)) 
        print('RMSLE:', RMSLE(y_test, y_pred))
        print('\n')
        return RMSLE(y_test, y_pred)
    else:
        return y_pred

In [38]:
rmse.append(random_forest(train_dataset, val_dataset, train_query_num, val_query_num))

Model 1. Random Forest

RMSLE: 0.05049120508951202




In [26]:
# Adaboost
def adaboost(X,X_test,y,y_test,validate=True):
    np.random.seed(100)
    y_pred = []
    model_clf = ensemble.AdaBoostRegressor(random_state=100, learning_rate = 0.1, n_estimators = 100)
#     hyper_params = {'learning_rate':[0.1, 0.5, 1.], 'n_estimators':[100, 200, 300]}
#     clf = model_selection.GridSearchCV(model_clf, hyper_params,cv=3,verbose=0, n_jobs=20)
#     clf.fit(X,y)
#     y_pred = clf.predict(X_test)
    model_clf.fit(X,y)
    y_pred = model_clf.predict(X_test)
    if validate:
        print('Model 2. Adaboost\n')
#         print('Best-parameters:{}'.format(clf.best_params_)) 
        print('RMSLE:', RMSLE(y_test, y_pred))
        print('\n')
        return RMSLE(y_test, y_pred)
    else:
        return y_pred

In [39]:
rmse.append(adaboost(train_dataset, val_dataset, train_query_num, val_query_num))

Model 2. Adaboost

RMSLE: 0.07420945485281821




In [40]:
from sklearn.svm import SVR
# SVM
def svm(X,X_test,y,y_test,validate=True):
    y_pred = []
    model_clf = SVR(kernel='linear',C=0.1, tol=0.001)
#     hyper_params = {'C':[0.1,1,10,25], 'tol':[0.0001, 0.001, 0.01]}
#     clf = model_selection.GridSearchCV(model_clf, hyper_params,cv=3,verbose=0, n_jobs=20)
#     clf.fit(X,y)
#     y_pred = clf.predict(X_test)
    model_clf.fit(X,y)
    y_pred = model_clf.predict(X_test)
    if validate:
        print('Model 3. SVM\n')
#         print('Best-parameters:{}'.format(clf.best_params_)) 
        print('RMSLE:', RMSLE(y_test, y_pred))
        print('\n')
        return RMSLE(y_test, y_pred)
    else:
        return y_pred


In [41]:
rmse.append(svm(train_dataset, val_dataset, train_query_num, val_query_num))

Model 3. SVM

RMSLE: 0.07039280940791545




In [42]:
from sklearn import linear_model
def Bayesian(X,X_test,y,y_test,validate=True):
    y_pred = []
    model_clf = linear_model.BayesianRidge(alpha_1 = 0.01, alpha_2 = 0.0001, lambda_1 = 0.0001, lambda_2 = 0.01)
#     hyper_params = {'alpha_1':[0.01,0.001,0.0001], 'alpha_2':[0.0001, 0.001, 0.01],'lambda_1':[0.0001, 0.001, 0.01],'lambda_2':[0.0001, 0.001, 0.01]}
#     clf = model_selection.GridSearchCV(model_clf, hyper_params,cv=3,verbose=0, n_jobs=20)
#     clf.fit(X,y)
#     y_pred = clf.predict(X_test)
    model_clf.fit(X,y)
    y_pred = model_clf.predict(X_test)
    if validate:
        print('Model 4. BayesianRidge\n')
#         print('Best-parameters:{}'.format(clf.best_params_)) 
        print('RMSLE:', RMSLE(y_test, y_pred))
        print('\n')
        return RMSLE(y_test, y_pred)
    else:
        return y_pred

In [43]:
rmse.append(Bayesian(train_dataset, val_dataset, train_query_num, val_query_num))

Model 4. BayesianRidge

RMSLE: 0.06755346530979488




In [44]:
from sklearn.linear_model import ElasticNet
def EN(X,X_test,y,y_test,validate=True):
    y_pred = []
    model_clf = ElasticNet(random_state=0, alpha = 1, l1_ratio = 0, tol = 0.0001)
#     hyper_params = {'alpha':[2,4,6,8,1], 'l1_ratio':[0,0.5], 'tol':[0.0001, 0.001, 0.01]}
#     clf = model_selection.GridSearchCV(model_clf, hyper_params,cv=3,verbose=0, n_jobs=20)
#     clf.fit(X,y)
#     y_pred = clf.predict(X_test)
    model_clf.fit(X,y)
    y_pred = model_clf.predict(X_test)
    if validate:
        print('Model 5. ElasticNet\n')
#         print('Best-parameters:{}'.format(clf.best_params_)) 
        print('RMSLE:', RMSLE(y_test, y_pred))
        print('\n')
        return RMSLE(y_test, y_pred)
    else:
        return y_pred

In [45]:
rmse.append(EN(train_dataset, val_dataset, train_query_num, val_query_num))

Model 5. ElasticNet

RMSLE: 0.19417097570997852




D:\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 179.1554699901585, tolerance: 0.04249751765001858
  positive)


In [20]:
from sklearn.ensemble import GradientBoostingRegressor
def GradientBoosting(X,X_test,y,y_test,validate=True):
    y_pred = []
    model_clf = GradientBoostingRegressor(random_state=0, learning_rate = 0.2, n_estimators = 800, min_samples_split = 6, min_samples_leaf = 4)
#     hyper_params = {'learning_rate':[0.2],'n_estimators':[800],'min_samples_split':[6],'min_samples_leaf':[3,4,5] }
#     clf = model_selection.GridSearchCV(model_clf, hyper_params,cv=3,verbose=0, n_jobs=20)
#     clf.fit(X,y)
#     y_pred = clf.predict(X_test)
    model_clf.fit(X,y)
    y_pred = model_clf.predict(X_test)
    if validate:
        print('Model 6. GradientBoosting\n')
#         print('Best-parameters:{}'.format(clf.best_params_)) 
        print('RMSLE:', RMSLE(y_test, y_pred))
        print('\n')
        return RMSLE(y_test, y_pred)
    else:
        return y_pred

In [47]:
rmse.append(GradientBoosting(train_dataset, val_dataset, train_query_num, val_query_num))

Model 6. GradientBoosting

RMSLE: 0.04648205584759347




In [48]:
from sklearn.neural_network import MLPRegressor
def MLP(X,X_test,y,y_test,validate=True):
    y_pred = []
    model_clf = MLPRegressor(random_state=1, max_iter=500, activation = 'tanh', solver = 'lbfgs', tol = 0.0001)
#     hyper_params = {'tol':[0.0001, 0.001, 0.01],'solver':['lbfgs', 'sgd', 'adam'],'activation':['identity', 'logistic', 'tanh', 'relu'] }
#     clf = model_selection.GridSearchCV(model_clf, hyper_params,cv=3,verbose=0, n_jobs=20)
#     clf.fit(X,y)
#     y_pred = clf.predict(X_test)
    model_clf.fit(X,y)
    y_pred = model_clf.predict(X_test)
    if validate:
        print('Model 7. MLPRegressor\n')
#         print('Best-parameters:{}'.format(clf.best_params_)) 
        print('RMSLE:', RMSLE(y_test, y_pred))
        print('\n')
        return RMSLE(y_test, y_pred)
    else:
        return y_pred    

In [49]:
rmse.append(MLP(train_dataset, val_dataset, train_query_num, val_query_num))

Model 7. MLPRegressor

RMSLE: 0.058226068230670065




D:\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [149]:
v2 = v
v2

[0.0636, 0.0772, 0.0717, 0.0681, 0.204, 0.0586, 0.0591]

In [186]:
v4 = v
v4

[0.0513, 0.0737, 0.0703, 0.0676, 0.1942, 0.0457, 0.0601]

In [74]:
from pyecharts import Bar

bar = Bar("RMSLE")
v = [round(x, 4) for x in rmse]
v1 = [0.093, 0.168, 0.169, 0.166, 0.211, 0.064, 0.086]
v2 = [0.0646, 0.0772, 0.0717, 0.0681, 0.204, 0.0586, 0.0591]
v3 = [0.0513, 0.0737, 0.0703, 0.0676, 0.1942, 0.0457, 0.0601]
# bar.add("查询计划(执行时间、输出行数、Filter平均值)",model_name, v3,title_pos='auto',mark_line=["min", "max"],width = 500, xaxis_rotate=15, is_label_show=True, xaxis_name_pos='end',xaxis_label_textsize=12)
# bar.add("查询计划（执行时间、输出行数、Filter）",model_name, v,title_pos='auto',width = 500, xaxis_rotate=15, is_label_show=False, xaxis_name_pos='end',xaxis_label_textsize=12)
# bar.add("查询计划(执行时间、输出行数)", model_name, v2,title_pos='auto',width = 500, xaxis_rotate=15, is_label_show=False, xaxis_name_pos='end',xaxis_label_textsize=12)
bar.add("仅查询数据", model_name, v1,title_pos='auto',bar_category_gap =  '40%',mark_line=["min", "max"],width = 500, xaxis_rotate=15, is_label_show=True, xaxis_name_pos='end',xaxis_label_textsize=12)
# bar.xAxis["axisLable"]['rotate'] = 45
# bar.set_global_opts(title_opts=opts.TitleOpts(title=title, subtitle=subtitle),xaxis_opts=opts.AxisOpts(name_rotate=60,name="123456",axislabel_opts={"rotate":45})
                    # bar.add("商家B", attr, v2, mark_line=["min", "max"])
# bar.show_config()
bar.render()

# 预测
注意：需修改模型函数，改成最佳参数，否则在测试时也使用gridsearch

In [22]:
train_dataset1, val_dataset1, test_dataset1, train_query_num1, val_query_num1, min_query_num, max_query_num, dicts, attr_min_max,max_cond_num, max_join_num = get_datasets(10000, train_val_ratio=1.0)

36244343 0
36244344 0
6
6


dump vectors

In [75]:
import json
json_train = json.dumps(train_dataset1.tolist(), ensure_ascii=False, indent=2)
with open('Vectors/train_vec.json', 'w') as json_file:
    json_file.write(json_train)

In [76]:
json_test = json.dumps(test_dataset1.tolist(), ensure_ascii=False, indent=2)
with open('Vectors/test_vec.json', 'w') as json_file:
    json_file.write(json_test)

In [86]:
json_query_num = json.dumps(train_query_num1.tolist(), ensure_ascii=False, indent=2)
with open('Vectors/query_num.json', 'w') as json_file:
    json_file.write(json_query_num)

In [77]:
import json
def read_json(filename):
    with open(filename, 'r') as f:
        dict_data = json.load(f)
        return dict_data

In [88]:
train_data = read_json('Vectors/query_num.json')
train_data

[0.6161091621267224,
 0.6868419149579457,
 0.7484043260031114,
 0.5774449069640016,
 0.5307199858668286,
 0.6897956382615174,
 0.7536095579581773,
 0.7313262400345834,
 0.6771948203434879,
 0.7060569615211424,
 0.02105811138959209,
 0.7684030033900701,
 0.6333830277533008,
 0.7575505018094836,
 0.8504251186553048,
 0.7037381660203333,
 0.7060697176472491,
 0.7232611550658633,
 0.7987964006335321,
 0.18823265813568923,
 0.8742740560694467,
 0.7877793798279222,
 0.5935261459237908,
 0.8393793219535988,
 0.7507334603533681,
 0.6771801425617967,
 0.6459686076469126,
 0.9243335229866876,
 0.21434558452045796,
 0.8184447158311433,
 0.7106839942092837,
 0.649839900882389,
 0.7841368745677236,
 0.45859378901564435,
 0.6411811198277062,
 0.5327313583334876,
 0.6941059326986351,
 0.6993033473820777,
 0.6923420655500615,
 0.7224775104388653,
 0.6411811198277062,
 0.6411811198277062,
 0.7297183934418263,
 0.841560176796032,
 0.9651592241590258,
 0.6990645025175569,
 0.718449668889022,
 0.513680682

In [145]:
y_pred = random_forest(train_dataset1, test_dataset1, train_query_num1, [], validate=False)
test_query_nums = denormalize_query_nums(y_pred, min_query_num, max_query_num)
file_name = "results/predictions_random_forest.csv"
os.makedirs(os.path.dirname(file_name), exist_ok=True)
with open(file_name, "w") as f:
    f.write("Query ID,Predicted Cardinality\n")
    for i in range(test_query_nums.shape[0]):
        if test_query_nums[i] < 0:
            test_query_nums[i] = 0
        f.write(str(i) + "," + str(test_query_nums[i]) + "\n")

In [161]:
y_pred = adaboost(train_dataset1, test_dataset1, train_query_num1, [], validate=False)
test_query_nums = denormalize_query_nums(y_pred, min_query_num, max_query_num)
file_name = "results/predictions_adaboost.csv"
os.makedirs(os.path.dirname(file_name), exist_ok=True)
with open(file_name, "w") as f:
    f.write("Query ID,Predicted Cardinality\n")
    for i in range(test_query_nums.shape[0]):
        if test_query_nums[i] < 0:
            test_query_nums[i] = 0
        f.write(str(i) + "," + str(test_query_nums[i]) + "\n")

In [167]:
y_pred = svm(train_dataset1, test_dataset1, train_query_num1, [], validate=False)
test_query_nums = denormalize_query_nums(y_pred, min_query_num, max_query_num)
file_name = "results/predictions_svm.csv"
os.makedirs(os.path.dirname(file_name), exist_ok=True)
with open(file_name, "w") as f:
    f.write("Query ID,Predicted Cardinality\n")
    for i in range(test_query_nums.shape[0]):
        if test_query_nums[i] < 0:
            test_query_nums[i] = 0
        f.write(str(i) + "," + str(test_query_nums[i]) + "\n")

In [23]:
y_pred = GradientBoosting(train_dataset1, test_dataset1, train_query_num1, [], validate=False)
test_query_nums = denormalize_query_nums(y_pred, min_query_num, max_query_num)
file_name = "results/predictions_GradientBoosting.csv"
os.makedirs(os.path.dirname(file_name), exist_ok=True)
with open(file_name, "w") as f:
    f.write("Query ID,Predicted Cardinality\n")
    for i in range(test_query_nums.shape[0]):
        if test_query_nums[i] < 0:
            test_query_nums[i] = 0
        f.write(str(i) + "," + str(test_query_nums[i]) + "\n")

In [191]:
y_pred = MLP(train_dataset1, test_dataset1, train_query_num1, [], validate=False)
test_query_nums = denormalize_query_nums(y_pred, min_query_num, max_query_num)
file_name = "results/predictions_MLP.csv"
os.makedirs(os.path.dirname(file_name), exist_ok=True)
with open(file_name, "w") as f:
    f.write("Query ID,Predicted Cardinality\n")
    for i in range(test_query_nums.shape[0]):
        if test_query_nums[i] < 0:
            test_query_nums[i] = 0
        f.write(str(i) + "," + str(test_query_nums[i]) + "\n")

D:\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
